# Get Your OM PKI Credential

## Only do this is you did NOT issue yourself an OM PKI credential as part of the OM PKI Course 

Ideally if you did not do this you should first take the full set of tutorials as it will give you a better idea of what this is all about. These can be found [here](https://github.com/OpenMined/PyDentity/tree/master/tutorials/5.%20OM%20FoPC%20Course%20-%20Public%20Key%20Infrastructures).

## 0. Install a Mobile Aries Agent of Your Choice

There are a number of apps available on the app store designed as *digital wallet's* to store verifiable credentials. The one's that work with the Hyperledger verifiable information platform all use code that implement's a set aries-rfcs, these specification's help ensure different implementations can interact.

You can pick the application you would prefer to install from any of the following: 

* [Connect.Me](https://www.connect.me/)
* [Trinsic](https://trinsic.id/)
* [eSatus](https://esatus.com/?lang=en)
* [LISSI](https://lissi.id/)

All can be a bit tempermental depending on your OS etc. If one isn't working for you try a different one. I have found eSatus to be the best for my old android phone.

Unfortunately to date none of the applications are open source, which in my view is leading to a fragmented experience for the individual. It is common to hear the phrase "Digital wallet's are the browsers of the future", or something along those lines. To which I often wonder why nobody has made it easy for me to write the equvalent of a webpage. 

Digital wallet's are an interesting space to keep an eye on. For the interested I recommend this report on [The Current and Future State of Digital Wallet's](https://wiki.hyperledger.org/download/attachments/24775535/The_Current_and_Future_State_of_Digital_Wallets_v1.0_FINAL.pdf?version=1&modificationDate=1574365535000&api=v2) from Darrell O'Donnell. Also available as a [webinar](https://ssimeetup.org/state-digital-identity-crypto-wallets-darrell-odonnell-webinar-22/).

The open source offering around mobile agent's and applications is gradually emerging, the most recent being the announcement of the [aries-bifold](https://github.com/Indicio-tech/aries-bifold) application and companion [aries-mobileagent-reactnative](https://github.com/Indicio-tech/aries-mobileagent-react-native) projects. If you would like to help us figure out how we could and should leverage and contribute too these offerings we would love to hear from you. 

For the moment we have to make do with what is available. So install one of the above applications and make sure you they are pointing to the **Sovrin StagingNet**. Different applications have different ways of changing the network the agent uses, usually in settings, although Connect.Me requires you select the network as part of the installation process.


## 1. Initialise a controller for Issuer

In [ ]:
%autoawait
import time
import os
import asyncio
from aries_cloudcontroller.aries_controller import AriesAgentController
from termcolor import colored,cprint

api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(
    f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}"
)
agent_controller = AriesAgentController(admin_url, api_key)

webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)
    

## 2. Register Listeners for Webhook Topics


In [ ]:
def cred_handler(payload):
    print("Handle Credential Webhook Payload")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    if state == "offer_sent":
        print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    print("Handle Connection Webhook Payload")
    global STATE
    connection_id = payload["connection_id"]
    print("Connection ID", connection_id)

    STATE = payload['state']
    print("State", STATE)
    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)


## 3. Establish a DIDComm connection with your Mobile Agent

Before you can issue yourself a credential using the OM Authority agent, you must establish a DIDComm connection by first exchanging DIDs. You can view the rfc for this protocol here.

You will first request the OM Authority agent to generate an invitation object containing a DID and assocaited public key. Then this information will need to be communicated **out of band**, the common approach to represent it as a QR Code that you will scan with your mobile SSI App. 

### 3a. Create an Invitation

The following cell creates the invitation object. This includes:
* `connection_id` - The primary key used to communicate to your agent which connection (and hence agent) you wish to send messages to
* `invitation` - A json object representing the invitation that needs to be communicated out of band. This includes the `recipientKeys` used to generate the DID and encrypt communication and a `seviceEndpoint` the location for the mobile agent to send messages too.
* `invitation_url` - This is the invitation object represented as a base64 string converted into a url. We use this to produce a QR Code.

Also notice the Connection webhook message output. This is from a webhook received by the AriesAgentController and handled by the function we defined in step 2. above.

In [ ]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
print("Connection ID", connection_id)
print("Invitation")
print(invite["invitation"])
print("Invite Url")
print(invite['invitation_url'])
inviteURL = invite['invitation_url']

### 3b. Generate QR Code and Scan

You should use the mobile application you installed at the start of the notebook.

Once you scan the QR Code and accept the connection on your mobile application you should see a series of Connection webhook payload messages output from the cell. This is the om-authority-agent letting the application know that it has recieved a series of messages associated with the did-exchange protocol from another agent.

Notice that the connection_id contained in this payload is the same everytime? This shows us that all the messages arrived across the same connection. This should be the same as the connection_id from the output above.

Notice also that each new payload we handle has a different state? This is the protocol progressing through it's different stages until it completes and the connection is moved to active. You can view the state machine for this protocol [here](https://github.com/hyperledger/aries-rfcs/tree/master/features/0023-did-exchange#state-machine-tables).

In [ ]:
import qrcode
# Link for connection invitation
input_data = inviteURL
# Creating an instance of qrcode
qr = qrcode.QRCode(
        version=1,
        box_size=10,
        border=5)
qr.add_data(input_data)
qr.make(fit=True)
img = qr.make_image(fill='black', back_color='white')
img.save('issuer_agent_invite_QRcode.png')

from IPython.display import Image
Image(width=400, filename='./issuer_agent_invite_QRcode.png')

### 3c. Check if established connection is in active state

Depending on the mobile application you chose to use the connection state may not have progress to active yet. You can usually fix this by sending a trust ping to this connection.

If the connection state is still not active then you may need to try another application, apologies.

In [ ]:
import time

# print('Current state for ConnectionId {} is {}'.format(connection_id,STATE))
print(colored("Current state for ConnectionId {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'active':
#     print('ConnectionId {0} is not in active state yet'.format(connection_id))
    print(colored("ConnectionId {0} is not in active state yet".format(connection_id), "yellow", attrs=["bold"]))
    trust_ping = await agent_controller.messaging.trust_ping(connection_id,'hello!')
#     print('Trust ping send to ConnectionId {0} to activate connection'.format(trust_ping))
    print(colored("Trust ping send to ConnectionId {0} to activate connection".format(trust_ping), "blue", attrs=["bold"]))
    time.sleep(5)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("ConnectionId: {0} is now active. Continue with notebook".format(connection_id), "green", attrs=["bold"]))


## 4. Write Credential Definition for OM PKI Schema


In [ ]:
pki_schema_id = "Sgg1wREgfEwbEPCQn9xEuE:2:OpenMined PKI Course:0.0.1"

cred_def_response = await agent_controller.definitions.write_cred_def(
    pki_schema_id
)
pki_cred_def_id = cred_def_response["credential_definition_id"]

if pki_schema_id and pki_cred_def_id:
    print("Successfully loaded identifiers required to issue OM PKI Course credentials")
    print("Schema :", pki_schema_id)
    print("Cred Def :", pki_cred_def_id)

## 5. Populate the Attribues for the Credential

Feel free to fill in your name however you like, this is your credential.

Note how information gets entered into a credential before it is offered, signed and accepted is a key process that should be thought through in relation to the context of the application. In many situations there will need to be high assurance that the information in a credential is reliable and accurate.

**Bad data in == Bad data out.**

In [ ]:
import datetime
tme = datetime.datetime.now()

name=input("Please enter the name you would like in your OM PKI Course Credential: ")
credential_attributes = [
    {"name": "name", "value": name},
    {"name": "date", "value": tme.strftime("%x")},
]
print(credential_attributes)

## 6. Send Credential

This should send a credential offer to your mobile application, once you accept the offer your mobile agent will blindly contribute a commitment to the agents master secret. The OM Authority will complete the signature on the credential and send it back to your mobile where you will store it.

Pay attention to the credential webhook messages that get output, these should track the protocol as it progresses through the different stages in the issue-credential flow. See the state diagram [here](https://github.com/hyperledger/aries-rfcs/blob/master/features/0036-issue-credential/credential-issuance.png).

There are other ways to issue a credential that require multiple api calls at each stage of the protocol.

**Arguments**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to False. **Not entirely sure about this one, believe its for logging. Also when set to True it throws an error**

In [ ]:
record = await agent_controller.issuer.send_credential(connection_id, pki_schema_id, pki_cred_def_id, credential_attributes, trace=False)


## 7. End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [ ]:
response = await agent_controller.terminate()
print(response)

# You should now have the necessary credential to authenticate against the OM Duet Authority Application. (Assuming it is running still)

See - http://139.162.224.50/
